#  Mental-RoBERTa
**Autora:** Antonia Estefane Ribeiro Veras

**Orientador:** Adonias Caetano de Oliveira

**Instituição:** IFCE

**Dataset disponível em:** https://www.kaggle.com/datasets/nikhileswarkomati/suicide-watch


## Instalação de Pacotes

In [ ]:
!pip install Unidecode

In [ ]:
!pip install wordcloud

In [ ]:
#hide
!pip install transformers

In [ ]:
!pip install lime

## Importação de Bibliotecas

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#text preprocessing libraries
import pandas as pd
import numpy as np
import math
import re
from unidecode import unidecode
from string import punctuation
import nltk
from nltk import sent_tokenize
from nltk import word_tokenize
from nltk.corpus import stopwords

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
#text classification libraries
from transformers import BertTokenizer, BertForSequenceClassification
import seaborn as sns
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from scipy.special import expit
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup
from torch.nn.utils import clip_grad_norm_
from tqdm.notebook import tqdm

In [ ]:
from lime.lime_text import LimeTextExplainer

## Carregamento do Dataset

In [ ]:
url = '(link púbico do Google Drive de acesso ao CSV de treinamento)'
file_id = url.split('/')[-2]
read_url='https://drive.google.com/uc?id=' + file_id

dataset = pd.read_csv(read_url,  index_col=0)
dataset = dataset[['selftext', 'is_suicide']]
dataset = dataset.rename(columns={'selftext': 'text', 'is_suicide': 'class'})

In [ ]:
data_process = dataset.copy()
dataset.head(10)

## Pré-Processamento de Texto

In [ ]:
nltk.download('rslp')
nltk.download('stopwords')
stopwords_list = stopwords.words("english")
print(stopwords_list)

In [ ]:
# Obtendo os textos originais
old_texts = data_process["text"]
new_texts = []

for text in old_texts:
    # Verifique se o texto não é um valor nulo (NaN)
    if isinstance(text, str):
        # Converte para minúsculas
        text = text.lower()
        # Remove menções (padrão do Twitter)
        text = re.sub('@[^\s]+', '', text)
        # Remove caracteres especiais do HTML
        text = re.sub('<[^<]+?>','', text)
        # Remove dígitos
        text = ''.join(c for c in text if not c.isdigit())
        # Remove URLs
        text = re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))', '', text)
        # Remove pontuação
        text = ''.join(c for c in text if c not in punctuation)
        # Remove stopwords
        text = ' '.join([word for word in text.split() if word not in stopwords_list])
        # Remove aspas duplas e simples
        text = text.replace("\"", "").replace("'", "")
        # Remove caracteres acentuados
        text = unidecode(text)
        # Adiciona o texto pré-processado à lista
        new_texts.append(text)
    else:
        # Se o texto for um valor nulo, você pode optar por ignorá-lo ou realizar outra ação adequada
        new_texts.append("")  # Aqui estou adicionando uma string vazia como um texto substituto

# Atualiza a coluna "text" com os textos pré-processados
data_process["text"] = new_texts


In [ ]:
data_process

## Visualização dos Dados

In [ ]:
sns.countplot(x = data_process['class'])

## Balanceamento dos Dados

In [ ]:
rus = RandomUnderSampler(random_state= 0)
X_bal, Y_bal = rus.fit_resample(data_process[['text']], data_process['class'])
sns.countplot(x = Y_bal)

## Criação da Wordcloud

In [ ]:
new_texts = data_process["text"]
all_words = ' '.join([text for text in new_texts])
word_cloud = WordCloud(width= 800, height= 500, max_font_size = 110, background_color="white", collocations = False).generate(all_words)
plt.figure(figsize=(20,10))
plt.imshow(word_cloud, interpolation='bilinear')
plt.axis("off")
plt.show()

## Divisão dos Dados em Conjuntos de Treinamento, Validação e Teste

In [ ]:
train_df, test_df, train_label, test_label = train_test_split(X_bal, Y_bal, test_size=0.20, random_state=42)

In [ ]:
train_df, valid_df, train_label,  valid_label = train_test_split(train_df, train_label, test_size=0.20, random_state=42)

In [ ]:
label_names = ['non-suicide', 'suicide']
label_names

## Inicialização do Tokenizador e do Modelo

In [ ]:
from transformers import BertForSequenceClassification
N_labels = len(train_label.unique())

PRETRAINED_LM = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_LM, do_lower_case=True)
tokenizer

## Definição de Funções Auxiliares

In [ ]:
def encode(docs):
    '''
    This function takes list of texts and returns input_ids and attention_mask of texts
    '''
    encoded_dict = tokenizer.batch_encode_plus(docs, add_special_tokens=True, max_length=128, padding='max_length',
                            return_attention_mask=True, truncation=True, return_tensors='pt')
    input_ids = encoded_dict['input_ids']
    attention_masks = encoded_dict['attention_mask']
    return input_ids, attention_masks

## Preparação dos dados para o treinamento do modelo

In [ ]:
train_input_ids, train_att_masks = encode(train_df['text'].values.tolist())
valid_input_ids, valid_att_masks = encode(valid_df['text'].values.tolist())
test_input_ids, test_att_masks = encode(test_df['text'].values.tolist())

In [ ]:
train_y = torch.LongTensor(train_label.values)
valid_y = torch.LongTensor(valid_label.values)
test_y = torch.LongTensor(test_label.values)
train_y.size(),valid_y.size(),test_y.size()

In [ ]:
BATCH_SIZE = 16
train_dataset = TensorDataset(train_input_ids, train_att_masks, train_y)
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=BATCH_SIZE)

valid_dataset = TensorDataset(valid_input_ids, valid_att_masks, valid_y)
valid_sampler = SequentialSampler(valid_dataset)
valid_dataloader = DataLoader(valid_dataset, sampler=valid_sampler, batch_size=BATCH_SIZE)

test_dataset = TensorDataset(test_input_ids, test_att_masks, test_y)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=BATCH_SIZE)

In [ ]:
train_label.unique()

In [ ]:
model = BertForSequenceClassification.from_pretrained(PRETRAINED_LM,
                                                      num_labels=N_labels,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
model = model.to(device)

In [ ]:
EPOCHS = 7
LEARNING_RATE = 2e-6

optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
scheduler = get_linear_schedule_with_warmup(optimizer,
             num_warmup_steps=0,
            num_training_steps=len(train_dataloader)*EPOCHS )

## Treinamento do modelo

In [ ]:
train_loss_per_epoch = []
val_loss_per_epoch = []


for epoch_num in range(EPOCHS):
    print('Epoch: ', epoch_num + 1)
    '''
    Training
    '''
    model.train()
    train_loss = 0
    for step_num, batch_data in enumerate(tqdm(train_dataloader,desc='Training')):
        input_ids, att_mask, labels = [data.to(device) for data in batch_data]
        output = model(input_ids = input_ids, attention_mask=att_mask, labels= labels)

        loss = output.loss
        train_loss += loss.item()

        model.zero_grad()
        loss.backward()
        del loss

        clip_grad_norm_(parameters=model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()

    train_loss_per_epoch.append(train_loss / (step_num + 1))


    '''
    Validation
    '''
    model.eval()
    valid_loss = 0
    valid_pred = []
    with torch.no_grad():
        for step_num_e, batch_data in enumerate(tqdm(valid_dataloader,desc='Validation')):
            input_ids, att_mask, labels = [data.to(device) for data in batch_data]
            output = model(input_ids = input_ids, attention_mask=att_mask, labels= labels)

            loss = output.loss
            valid_loss += loss.item()

            valid_pred.append(np.argmax(output.logits.cpu().detach().numpy(),axis=-1))

    val_loss_per_epoch.append(valid_loss / (step_num_e + 1))
    valid_pred = np.concatenate(valid_pred)

    '''
    Loss message
    '''
    print("{0}/{1} train loss: {2} ".format(step_num+1, math.ceil(len(train_df) / BATCH_SIZE), train_loss / (step_num + 1)))
    print("{0}/{1} val loss: {2} ".format(step_num_e+1, math.ceil(len(valid_df) / BATCH_SIZE), valid_loss / (step_num_e + 1)))

In [ ]:
from matplotlib import pyplot as plt

epochs = range(1, EPOCHS +1 )
fig, ax = plt.subplots()
ax.plot(epochs,train_loss_per_epoch,label ='training loss')
ax.plot(epochs, val_loss_per_epoch, label = 'validation loss' )
ax.set_title('Training and Validation loss')
ax.set_xlabel('Epochs')
ax.set_ylabel('Loss')
ax.legend()
plt.show()

In [ ]:
model.eval()
test_pred = []
test_loss= 0

with torch.no_grad():
    for step_num, batch_data in tqdm(enumerate(test_dataloader)):
        input_ids, att_mask, labels = [data.to(device) for data in batch_data]
        output = model(input_ids = input_ids, attention_mask=att_mask, labels= labels)

        loss = output.loss
        test_loss += loss.item()

        test_pred.append(np.argmax(output.logits.cpu().detach().numpy(),axis=-1))
test_pred = np.concatenate(test_pred)

## Avaliação do Modelo

In [ ]:
from sklearn.metrics import classification_report
print('classification report')
print(classification_report(test_pred, test_label.to_numpy(),target_names=label_names))


In [ ]:
from scipy.special import expit

model.eval()
probabilities = []

with torch.no_grad():
  for step_num, batch_data in tqdm(enumerate(test_dataloader)):
      input_ids, att_mask, labels = [data.to(device) for data in batch_data]
      output = model(input_ids = input_ids, attention_mask=att_mask, labels= labels)

      prob = expit(output.logits.cpu().detach().numpy())

      probabilities.append( prob )

probabilities = np.concatenate(probabilities)

In [ ]:
probabilities

In [ ]:
from sklearn.metrics import roc_curve, auc

def plot_roc_curve(fper, tper, AUC):
    plt.plot(fper, tper, color="red", label=f"AUC = {AUC}")
    plt.plot([0, 1], [0, 1], color="green", linestyle="--")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("Receiver Operating Characteristic Curve")
    plt.legend()
    plt.show()


fper, tper, thresholds = roc_curve(test_label.to_numpy(), test_pred.to_numpy())
AUC = auc(fper, tper)
plot_roc_curve(fper, tper, AUC)

In [ ]:
fper, tper, thresholds = roc_curve(test_label.to_numpy(), probabilities[::,1])
AUC = auc(fper, tper)
plot_roc_curve(fper, tper, AUC)

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
def plot_confusion_matrix(y_preds, y_true, labels=None):
  cm = confusion_matrix(y_true, y_preds, normalize="true")
  fig, ax = plt.subplots(figsize=(6, 6))
  disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
  disp.plot(cmap="Blues", values_format=".2f", ax=ax, colorbar=False)
  plt.title("Normalized confusion matrix")
  plt.show()

In [ ]:
plot_confusion_matrix(test_pred, test_label.to_numpy())

In [ ]:
test_df['pred'] = test_pred
test_df['label'] = test_label
test_df['prob'] = probabilities
test_df.reset_index(level=0)
print(test_df[test_df['label']!=test_df['pred']].shape)
test_df[test_df['label']!=test_df['pred']][['text','label','pred']].head(10)

## **Saving results**

In [ ]:
import shutil

In [ ]:
#hide
test_df.to_csv("test_results_BERTimbauLarge.csv",index= False)

In [ ]:
shutil.copy('/content/test_results_BERTimbauLarge.csv', '/content/drive/MyDrive/test_results_BERTimbauLarge.csv')

In [ ]:
MODEL_PATH = "BERTimbauLarge.bin"
torch.save(model.state_dict(), MODEL_PATH)

In [ ]:
shutil.copy('/content/BERTimbauLarge.bin', '/content/drive/MyDrive/BERTimbauLarge.bin')

## Interpretabilidade do Modelo com Lime

In [ ]:
def createDataloader(text):
  test_input_ids, test_att_masks = encode([text])
  BATCH_SIZE = 16
  test_y = torch.LongTensor([0])
  test_dataset = TensorDataset(test_input_ids, test_att_masks, test_y)
  test_sampler = SequentialSampler(test_dataset)
  test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=BATCH_SIZE)
  return test_dataloader

In [ ]:
def predict(text, labels_names):
    model.eval()
    test_dataloader = createDataloader(text)

    with torch.no_grad():
        for step_num, batch_data in tqdm(enumerate(test_dataloader)):
            input_ids, att_mask, labels = [data.to(device) for data in batch_data]
            output = model(input_ids=input_ids, attention_mask=att_mask, labels=labels)

            logits = output.logits.cpu().detach().numpy()
            index_pred = np.argmax(logits, axis=-1)[0]
            probabilities = expit(logits)[0]

    df = pd.DataFrame(columns=['Label', 'Probabilidade'])
    df['Label'] = labels_names
    df['Probabilidade'] = probabilities

    return index_pred, labels_names[index_pred], df

In [ ]:
labels_names = ['non-suicide', 'suicide']
text = 'think better night posts pretty shit shitposts rn like 7 ish hours making shitty posts sorry'
index_pred, label, df = predict(text, labels_names)

In [ ]:
def addlabels(x, y):
    for i in range(len(x)):
        plt.text(y[i], i, str(f'{y[i]*100:0.2f}%'), ha='center', bbox=dict(facecolor='blue', alpha=.6))

plt.clf()

pos = np.arange(len(df['Label'].values))
probabilidades = df['Probabilidade'].values

# Definir as cores para cada classe
colors = df['Label'].map({'non-suicide': 'g', 'suicide': 'r'})

# Criar o gráfico de barras
fig = plt.barh(pos, probabilidades, color=colors, edgecolor='black')
plt.yticks(pos, df['Label'])  # Exibir cada classe no eixo y
plt.legend(fig, [str(i) for i in ['Negativo', 'Positivo']])  # Exibir a legenda de cada classe (Positivo ou Negativo)
plt.xlabel('Probabilidades', fontsize=16)
plt.ylabel('Classe', fontsize=16)
addlabels(pos, probabilidades)  # Chamar a função para adicionar rótulos de valores
plt.title('Probabilidades de Predição', fontsize=16)
plt.show()


In [ ]:
def predict_proba(sentences):
  model.eval()
  probabilities = []

  test_input_ids, test_att_masks = encode(sentences)
  BATCH_SIZE = 16
  test_y = torch.LongTensor([0] * len(sentences))
  test_dataset = TensorDataset(test_input_ids, test_att_masks, test_y)
  test_sampler = SequentialSampler(test_dataset)
  test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=BATCH_SIZE)

  with torch.no_grad():
    for step_num, batch_data in tqdm(enumerate(test_dataloader)):
        input_ids, att_mask, labels = [data.to(device) for data in batch_data]
        output = model(input_ids = input_ids, attention_mask=att_mask, labels= labels)

        probabilities.append(expit(output.logits.cpu().detach().numpy()))

  probabilities = np.concatenate(probabilities)

  return probabilities

In [ ]:
labels_names = ['non-suicide', 'suicide']

In [ ]:
explainer = LimeTextExplainer(class_names = labels_names)

In [ ]:
exp = explainer.explain_instance('know 7 months self harm free urge gets stronger stronger every day past days shit family stresses especially nephew 12 mum full custody told go kill numerous times wish easy partners family stresses especially past days stress trigger want scream cry live near woods could go scream till ca possibly scream anymore cops probably get called oh knows depression really know feeling moment would want come make sure ok fucking annoying annoys time small stupid things even know going post', classifier_fn = predict_proba, num_features = 10)

In [ ]:
exp.show_in_notebook(text = True)

In [ ]:
from IPython.core.display import display, HTML
display(HTML(exp.as_html()))

In [ ]:
exp = explainer.explain_instance('feeling depressed stressed someone actually caring talk feel like dearth caring affectionate people planet everyone terrible indifferent write paragraphs paragraphs text make someone feel important never get return', classifier_fn = predict_proba, num_features = 10)

In [ ]:
exp.show_in_notebook(text = True)

In [ ]:
from IPython.core.display import display, HTML
display(HTML(exp.as_html()))

In [ ]:
def execute_classify(input):

  if model != None:
    test_pred = predict(input)
    label = labels_names[test_pred[0]]
    return f"A classe prevista foi: {label}"
  else:
    return "Não há texto ou modelo para classificação!"

In [ ]:
exp.as_pyplot_figure()

In [ ]:
from google.colab import output
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')